## 3.1 Preparation

### Import

In [1]:
import pymysql
import pymongo
from pymongo import MongoClient

import pandas as pd
import time # add timestamps to cache entries
import json # serialize/deserialize data when saving/loading the cache to/from disk
import threading # call the checkpoint() method periodically

In [2]:
import os
os.getcwd()

'/Users/cyan/Documents/python_workspace/694_DBMngm_23SP'

### Connecting to MongoDB

In [34]:
# Connect to the MongoDB database
client = MongoClient('localhost', 27017)
db = client['694db_nsdb']
tweets = db['twt_datetime']

#### test for tweets load

In [4]:
documents = tweets.find().limit(5)
pd.DataFrame(documents)

,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,644b36afa1b284129a82b31b,"{'id': 301470336, 'id_str': '301470336', 'name...","ahap , low cut off merchants","{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254022778371571712,None,None,None,1254016297161105408,0,0,0,0,en
1,644b36afa1b284129a82b318,"{'id': 375777294, 'id_str': '375777294', 'name...",im making 17 a hr doing nothing 😭😭😭😭 i love th...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254022776094105602,None,None,None,None,0,0,0,0,en
2,644b36afa1b284129a82b316,"{'id': 2929344220, 'id_str': '2929344220', 'na...",Was sollen 150 Euro Computerzuschuss bringen? ...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022772575043586,None,None,None,1254016434658848769,0,0,0,0,de
3,644b36afa1b284129a82b31a,"{'id': 1132273796138905600, 'id_str': '1132273...",Oh brother and he gave the kid his corona type...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022776752615430,None,None,None,None,0,0,0,0,en
4,644b36afa1b284129a82b317,"{'id': 1091660129894838272, 'id_str': '1091660...",@VinceMcMahon @TripleH We hereby honor to anno...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022773598572544,VinceMcMahon,1222639789,1253768179517657088,None,0,0,0,0,en


### Connecting to MySQL

In [5]:
#Connect to MySQL
conn = pymysql.connect(host='localhost', port=3306, user='root',password="led08854",charset="utf8mb4", database='694RDBMS')
cur = conn.cursor()

In [6]:
cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users;")
users = pd.DataFrame(cur.fetchall())
users.columns = [desc[0] for desc in cur.description]
users.head(10)

,userid_str,screen_name,name,favourites_count
0,1000006582896295938,yourgirlsarag,sara,11349
1,1000027886915637250,cheche04125843,cheche,1031
2,1000034375973646337,clarice8967,clarih ?,40141
3,100004211,karundhiravidan,azakiya tamilmagan,70330
4,1000045322859634688,barcelona9581,barcelona958,29449
5,1000046756120363013,isisevang,áisis,9384
6,1000047984103575552,esesesrarara34,EsRaAa (EN YENI HESAP)?,1459
7,1000055605263065088,zolesa_ke,Tsika Zolesa,37216
8,1000061119921352705,lejayk54,☮️Axel☮️,1438
9,1000062434126725120,mahin37418807,Mahin Mohades,5232


### Cache
Requirements  
- Design and implement a cache for storing "popular" (frequently accessed) data so that this data does not have to be retrieved from the database each time it is accessed.   
    - Some hashtags/users may be popular and their data may be cached.   
- You can use a Python dictionary for implementing the cache, but you must :
    - limit the size of the dictionary by evicting entries using a strategy (E.g. least accessed). 
    - You must checkpoint your data on disk at periodic intervals. 
    - When your search application starts up, you must reload the state of the cache from the disk. 
- Questions to consider 
    - Can an entry in the cache get stale (is not representing the correct state)? 
    - How will you update or purge stale data? 
    - An advanced feature that you could implement is an expiry mechanism for an entry in the cache by having a Time-To-Live field for each entry that determines the amount of time the entry will be retained in the cache. 
- Timings of your test search queries (make sure you are hitting cached and non cached data)

#### Cache.json content initiated

In [7]:
# store the 100 most popular user info into cache.json
cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users order by favourites_count desc limit 100") # there are 80,943 entries in MySQL in total
cache = pd.DataFrame(cur.fetchall())
cache.columns = [desc[0] for desc in cur.description]
cache['name_as_index']=cache['screen_name']
cache['access_count']=0

# get the wanted subset
subset_data = cache[["userid_str", "screen_name", "name", "favourites_count", "name_as_index"]]
subset_access_count = cache[["screen_name", "access_count", "name_as_index"]]

# convert df to dict
cache_data_dict = subset_data.set_index("name_as_index").to_dict("index")
cache_access_count_dict = subset_access_count.set_index("name_as_index").to_dict("index")

# combine into cache_dict
cache_dict = {}

# set keys
key1 = "data"
key2 = "access_count"
cache_dict = {key1: cache_data_dict, key2: cache_access_count_dict}

In [8]:
# Write the dictionary to a JSON file
with open("cache.json", "w") as f:
    json.dump(cache_dict, f)

#### Define the Cache Class

In [9]:
class Cache:
    def __init__(self, max_size = 1000, eviction_strategy = "least_accessed", checkpoint_interval = 8, filename = "cache.json"):
        self.max_size = max_size
        self.eviction_strategy = eviction_strategy
        self.checkpoint_interval = checkpoint_interval
        self.filename = filename
        self.data = {}
        self.access_count = {}
        # laod the cache from the file
        self.load() 
        
        # Start a background thread to periodically checkpoint the cache
        self.checkpoint_thread = threading.Thread(target=self.periodic_checkpoint)
        self.checkpoint_thread.daemon = True
        self.checkpoint_thread.start()

    def get(self, key):
        # Get an item from the cache and update access count
        self.access_count[key]["access_count"] += 1
        return self.data[key]

    def set(self, key, value):
        # Add an item to the cache and evict least popular item if max_size is reached
        self.data[key] = value
        self.access_count[key] = {'screen_name': key, 'access_count': 1} # access 1 time
        if len(self.data) > self.max_size:
            self.evict()

    def __contains__(self, key):
        return key in self.data

    def evict(self):
        # Evict the least popular item from the cache
        if self.eviction_strategy == "least_accessed":
            key_to_evict = min(self.access_count, key=self.access_count.get)
        else:
            # Other eviction strategies can be implemented here
            pass
        del self.data[key_to_evict]
        del self.access_count[key_to_evict]
        
    def load(self):
        # Load the cache from disk
        with open(self.filename, "r") as f:
            cache_data = json.load(f)
            self.data = cache_data["data"]
            self.access_count = cache_data["access_count"]
        
    def periodic_checkpoint(self):
        while True:
            # Wait for the checkpoint interval
            time.sleep(self.checkpoint_interval)
            
            # Checkpoint the cache to the file
            self.checkpoint()

    def checkpoint(self):
        # Save the cache to disk
        with open(self.filename, "w") as f:
            json.dump({
                "data": self.data,
                "access_count": self.access_count
            }, f)
    
    def is_stale(self, key, ttl = 60*60):
        # Check if an item in the cache is stale (i.e. expired)
        if key not in self.access_count:
            return True
        return time.time() - self.access_count[key] > ttl
    
    def update_or_purge_stale_data(self, ttl):
        # Update or purge stale data in the cache
        keys_to_purge = []
        for key in self.data.keys():
            if self.is_stale(key, ttl):
                keys_to_purge.append(key)
        for key in keys_to_purge:
            del self.data[key]
            del self.access_count[key]

    def __del__(self):
        # Save the cache to disk before exiting
        self.checkpoint()

In [10]:
# initialize the cache
cache = Cache(
    max_size=1000,
    eviction_strategy="least_accessed",
    checkpoint_interval = 8, # checkpoint every 8 seconds
    filename="cache.json"
)

In [11]:
# load cache from disk
cache.load()

#### example usage & test

##### get test

In [298]:
user_id = "ronniehowlett3"
if user_id in cache:
    # cache hit: get user data from cache directly
    user_data = pd.DataFrame.from_dict(cache.get(user_id), orient = "index").T
    print("in cache")
else:
    print("not in cache")
    # cache miss: get data from database
    cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;", (user_id,))
    user_data = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
    # Add user data to cache
    user_data_dict = user_data.to_dict("index")[0]
    cache.set(user_id, user_data_dict)

in cache


In [299]:
user_data

,userid_str,screen_name,name,favourites_count
0,752559084881211392,ronniehowlett3,ronnie cassol,1105807


In [300]:
# memory test
cache.access_count[user_id]

{'screen_name': 'ronniehowlett3', 'access_count': 9}

In [301]:
# disk test
with open("cache.json", "r") as f:
    # Load the JSON data into a dictionary
    cache_disk = json.load(f)
    
cache_disk["access_count"].get(user_id)

{'screen_name': 'ronniehowlett3', 'access_count': 9}

##### set test

In [305]:
# set test
user_id = "Sm03leBr00t"
if user_id in cache:
    # cache hit: get user data from cache directly
    user_data = pd.DataFrame.from_dict(cache.get(user_id), orient = "index").T
    print("in cache")
else:
    print("not in cache")
    # cache miss: get data from database
    cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;", (user_id,))
    user_data = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
    # Add user data to cache
    user_data_dict = user_data.to_dict("index")[0]
    cache.set(user_id, user_data_dict)
    # del cache.data[user_id]
    # del cache.access_count[user_id]

not in cache


In [306]:
user_data

,userid_str,screen_name,name,favourites_count
0,994069987135541248,Sm03leBr00t,Sm03leBr00t,0


In [307]:
# memory test
cache.access_count[user_id]

{'screen_name': 'Sm03leBr00t', 'access_count': 1}

In [308]:
# disk test
with open("cache.json", "r") as f:
    # Load the JSON data into a dictionary
    cache_disk = json.load(f)
    
cache_disk["access_count"].get(user_id)

{'screen_name': 'Sm03leBr00t', 'access_count': 1}

## 3.2 Single Queries in NRDB for the search application

### 3.2.1 Search by keyword

In [12]:
#Find tweets with keyword
tweets.create_index([("text", pymongo.TEXT)])
def search_word():
    user_input = input("Please enter a keyword: ")
    # search in NRDB
    if user_input != "":
        myquery = {"$text":{"$search": user_input}}
        twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(users, twts, on='userid_str')
    return testItems

In [13]:
search_word()

Please enter a keyword:  RT


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,1135583492702687234,wasthend,William Almeida,935,644b36e1a1b284129a834476,"{'id': 1135583492702687234, 'id_str': '1135583...",Por favor deem Rt e denunciem.,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254056652610945025,None,None,None,1254038813195800576,0,0,0,0,pt
1,1149671626616610817,PrabhakarKolla2,Prabhakar Kollapudi,945,644b36cea1b284129a8314ae,"{'id': 1149671626616610817, 'id_str': '1149671...",Without tabilighi india would have been corona...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254045968443437056,None,None,None,None,0,0,0,0,en
2,1221778286700679170,naolanic,BELI ROTI KERING NAO DONG CEK PINNED/GI LIMIT SBR,2952,644b36b9a1b284129a82d13e,"{'id': 1221778286700679170, 'id_str': '1221778...",@stmaemunah05 @radenrauf https://t.co/5WZm9h1c...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254030251157413888,stmaemunah05,2215623222,1254030070475141121,1250610299004055554,0,0,0,0,in
3,1221778286700679170,naolanic,BELI ROTI KERING NAO DONG CEK PINNED/GI LIMIT SBR,2952,644b36cca1b284129a830b99,"{'id': 1221778286700679170, 'id_str': '1221778...",@GRESAIDS https://t.co/gOJ8ltsonR bantu RT Kak...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254043945933262851,GRESAIDS,1101162508583223297,1254018889366683648,1250610299004055554,0,0,0,0,in
4,1366789854,astapidrullah,.pid,11273,644b36bea1b284129a82e212,"{'id': 1366789854, 'id_str': '1366789854', 'na...",Bacot lo mas mas sirkel rt.,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254034190070214656,None,None,None,1253939460078592000,0,0,0,0,in
5,2306048802,OgiMontena_,HeyJude,1477,644b36b0a1b284129a82b7c1,"{'id': 2306048802, 'id_str': '2306048802', 'na...",Oh sirkel rt ckck pantes kek kontol,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254023975379677185,None,None,None,1253939460078592000,0,0,0,0,in
6,2914383445,manameismarshal,???????,358,644b36cda1b284129a831052,"{'id': 2914383445, 'id_str': '2914383445', 'na...",Anjay ada yang positif corona di rt aku,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254045023278002176,None,None,None,None,0,0,0,0,in
7,3950306362,ViihMss15,Vi Gabs,882,644b36e4a1b284129a834abe,"{'id': 3950306362, 'id_str': '3950306362', 'na...",Dêem rt e denunciem,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254058043282784256,None,None,None,1254038813195800576,0,0,0,0,pt
8,632834693,ivey00880866,ivey,23607,644b36daa1b284129a833841,"{'id': 632834693, 'id_str': '632834693', 'name...",RT,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254053961780387840,None,None,None,1251938514670948354,0,0,0,0,und
9,979285512040009728,Thaladhoniabar1,Thaladhoniveriyan?,4509,644b36bca1b284129a82dd32,"{'id': 979285512040009728, 'id_str': '97928551...",#அன்புள்ளசூர்யா Rt potavanuku... Corona varath...,"{'hashtags': [{'text': 'அன்புள்ளசூர்யா', 'indi...",2020-04-25,1254033090571165698,None,None,None,None,0,0,0,0,und


### 3.2.2 Search by hashtag

In [14]:
#Find tweets with hastage
def search_hashtag():
    user_input = input("Please enter a hashtag: ")
    if user_input !="":
        myquery = {"entities.hashtags.text": user_input}
        twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(users, twts, on='userid_str')
    return testItems

In [15]:
search_hashtag()

Please enter a hashtag:  sport


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,2242948745,tho1965,Thomas Krause,2184,644b36afa1b284129a82b315,"{'id': 2242948745, 'id_str': '2242948745', 'na...",Schöne Runde mit dem Rennrad ✌️\n#sport #coron...,"{'hashtags': [{'text': 'sport', 'indices': [32...",2020-04-25,1254022770746372096,None,None,None,None,0,0,0,0,de


### 3.2.3 Search by user name

In [16]:
def search_user_name():
    user_input = input("Please enter a user_name: ")
    if user_input !="":
        # my sql
        myquery = "SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;"
        cur.execute(myquery,(user_input,))
        uers = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
        # mongodb
        userid_str=uers["userid_str"].tolist()
        myquery = {"user.id_str": {"$in":userid_str}}
        twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
        #Processing
        twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
        #JOIN
        testItems = pd.merge(users, twts, on='userid_str')
        return testItems

In [17]:
search_user_name()

Please enter a user_name:  tho1965


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,2242948745,tho1965,Thomas Krause,2184,644b36afa1b284129a82b315,"{'id': 2242948745, 'id_str': '2242948745', 'na...",Schöne Runde mit dem Rennrad ✌️\n#sport #coron...,"{'hashtags': [{'text': 'sport', 'indices': [32...",2020-04-25,1254022770746372096,None,None,None,None,0,0,0,0,de


### 3.2.4 Search by time range

In [18]:
def search_date():
    myquery={}
    start_at = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
    end_at = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
    if start_at and end_at != "":
        myquery['created_at'] = {'$gte': start_at, '$lte': end_at}
    elif start_at != "":
        myquery['created_at'] = {'$gte': start_at}
    elif end_at != "":
        myquery['created_at'] = {'$lte': end_at}
    twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    return twts

In [20]:
search_date()

Please enter a start date(format:yyyy-mm-dd hh:mm:ss):  2020-04-20
Please enter a end date(format:yyyy-mm-dd hh:mm:ss):  2020-04-20


""


## 3.3 Search Function by all types

#### 3.3.1 using no cache

In [95]:
def search_all_RNR():
    myquery={}
    search_keyword = input("Please enter a keyword: ")
    search_hashtag = input("Please enter a hashtag: ")
    search_user = input("Please enter a user_name: ")
    start_at = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
    end_at = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
    # screen name
    if search_user != "":
        # my sql
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;", (search_user,))
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
        # mongodb
        userid_str = uer["userid_str"].tolist()
        myquery["user.id_str"] = {"$in":userid_str}
    else:
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users;")
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
    if search_keyword != "":
        myquery["$text"] = {"$search": search_keyword}
    if search_hashtag != "":
        myquery["entities.hashtags.text"] = search_hashtag
    # time range
    if start_at and end_at != "":
        myquery['created_at'] = {'$gte': start_at, '$lte': end_at}
    elif start_at != "":
        myquery['created_at'] = {'$gte': start_at}
    elif end_at != "":
        myquery['created_at'] = {'$lte': end_at}
    twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(uer, twts, on='userid_str')
    return testItems

###### take a view

In [96]:
search_all_RNR()

Please enter a keyword:  
Please enter a hashtag:  
Please enter a user_name:  
Please enter a start date(format:yyyy-mm-dd hh:mm:ss):  
Please enter a end date(format:yyyy-mm-dd hh:mm:ss):  


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,1091660129894838272,milliteknoloj,Milli teknoloji,352,644b36afa1b284129a82b317,"{'id': 1091660129894838272, 'id_str': '1091660...",@VinceMcMahon @TripleH We hereby honor to anno...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022773598572544,VinceMcMahon,1222639789,1253768179517657088,None,0,0,0,0,en
1,1120761000561606656,cloudseokjinie,?,1957,644b36afa1b284129a82b31e,"{'id': 1120761000561606656, 'id_str': '1120761...",tony montana yoongi live #kpop trump bp lisa d...,"{'hashtags': [{'text': 'kpop', 'indices': [25,...",2020-04-25,1254022781710274566,None,None,None,None,0,0,0,0,en
2,1132273796138905600,RampTheresa,Terri Kamp,29166,644b36afa1b284129a82b31a,"{'id': 1132273796138905600, 'id_str': '1132273...",Oh brother and he gave the kid his corona type...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022776752615430,None,None,None,None,0,0,0,0,en
3,2242948745,tho1965,Thomas Krause,2184,644b36afa1b284129a82b315,"{'id': 2242948745, 'id_str': '2242948745', 'na...",Schöne Runde mit dem Rennrad ✌️\n#sport #coron...,"{'hashtags': [{'text': 'sport', 'indices': [32...",2020-04-25,1254022770746372096,None,None,None,None,0,0,0,0,de
4,2929344220,RusticusArat,Ralf Schmitz,32024,644b36afa1b284129a82b316,"{'id': 2929344220, 'id_str': '2929344220', 'na...",Was sollen 150 Euro Computerzuschuss bringen? ...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022772575043586,None,None,None,1254016434658848769,0,0,0,0,de
5,301470336,_FreshAA,BABA ALI,857,644b36afa1b284129a82b31b,"{'id': 301470336, 'id_str': '301470336', 'name...","ahap , low cut off merchants","{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254022778371571712,None,None,None,1254016297161105408,0,0,0,0,en
6,375777294,TWD40_,TeéLaneeë?,1325,644b36afa1b284129a82b318,"{'id': 375777294, 'id_str': '375777294', 'name...",im making 17 a hr doing nothing 😭😭😭😭 i love th...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254022776094105602,None,None,None,None,0,0,0,0,en
7,804046791348015107,B_King69,Bi Sex Uau,5446,644b36afa1b284129a82b314,"{'id': 804046791348015107, 'id_str': '80404679...","É isto, ou vou morrer sem ar ou com o corona h...","{'hashtags': [], 'urls': [], 'user_mentions': ...",2020-04-25,1254022770679320576,None,None,None,None,0,0,0,0,pt
8,865733987561381888,BritishKatieKim,ANH,244132,644b36afa1b284129a82b319,"{'id': 865733987561381888, 'id_str': '86573398...",@MichaelTouby @ChinaDaily @RVsmtown @YRFairyta...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022776207429633,MichaelTouby,719625570615758849,1254013543470563328,None,0,0,0,0,en
9,923169415054680064,QuantumAspect,Quantum,119803,644b36afa1b284129a82b31d,"{'id': 923169415054680064, 'id_str': '92316941...",Weekly mortality graphs show in some European ...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",2020-04-25,1254022780695252993,None,None,None,None,0,0,0,0,en


#### 3.3.2 using cache

In [14]:
def search_all_C():
    myquery={}
    search_keyword = input("Please enter a keyword: ")
    search_hashtag = input("Please enter a hashtag: ")
    search_user = input("Please enter a user_name: ")
    start_at = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
    end_at = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
    # screen name
    if search_user != "":
        # my sql
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;", (search_user,))
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
        # mongodb
        userid_str = uer["userid_str"].tolist()
        myquery["user.id_str"] = {"$in":userid_str}
    else:
        uer = users
    if search_keyword != "":
        myquery["$text"] = {"$search": search_keyword}
    if search_hashtag != "":
        myquery["entities.hashtags.text"] = search_hashtag
    # time range
    if start_at and end_at != "":
        myquery['created_at'] = {'$gte': start_at, '$lte': end_at}
    elif start_at != "":
        myquery['created_at'] = {'$gte': start_at}
    elif end_at != "":
        myquery['created_at'] = {'$lte': end_at}
    twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(uer, twts, on='userid_str')
    return testItems

###### take a view

In [15]:
search_all_C()

Please enter a keyword: 
Please enter a hashtag: 
Please enter a user_name: tho1965
Please enter a start date(format:yyyy-mm-dd hh:mm:ss): 
Please enter a end date(format:yyyy-mm-dd hh:mm:ss): 


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,2242948745,tho1965,Thomas Krause,2184,644aeb6165c7444702bc28fd,"{'id_str': '2242948745', 'created_at': '2013-1...",Schöne Runde mit dem Rennrad ✌️\n#sport #coron...,"{'hashtags': [{'text': 'sport', 'indices': [32...",2020-04-25,1254022770746372096,None,None,None,None,0,0,0,0,de


## 3.4 time test

#### for uncached data

In [35]:
def test_uncached_performance():
    myquery={}
    search_keyword = input("Please enter a keyword: ")
    search_hashtag = input("Please enter a hashtag: ")
    search_user = input("Please enter a user_name: ")
    start_at = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
    end_at = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
    # search start
    start_time = time.time()
    # screen name
    if search_user != "":
        # my sql
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;", (search_user,))
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
        # mongodb
        userid_str = uer["userid_str"].tolist()
        myquery["user.id_str"] = {"$in":userid_str}
    else:
        cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users;")
        uer = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
    if search_keyword != "":
        myquery["$text"] = {"$search": search_keyword}
    if search_hashtag != "":
        myquery["entities.hashtags.text"] = search_hashtag
    # time range
    if start_at and end_at != "":
        myquery['created_at'] = {'$gte': start_at, '$lte': end_at}
    elif start_at != "":
        myquery['created_at'] = {'$gte': start_at}
    elif end_at != "":
        myquery['created_at'] = {'$lte': end_at}
    twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(uer, twts, on='userid_str')
    # search end
    end_time = time.time()
    print(f''' searching in cached information took {end_time - start_time:.4f} seconds''')
    return testItems

In [52]:
test_uncached_performance() # first time

Please enter a keyword:  
Please enter a hashtag:  
Please enter a user_name:  TomthunkitsMind
Please enter a start date(format:yyyy-mm-dd hh:mm:ss):  
Please enter a end date(format:yyyy-mm-dd hh:mm:ss):  


 searching in cached information took 0.1215 seconds


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,289118612,TomthunkitsMind,Tomthunkit™,22437,644b36b1a1b284129a82bae5,"{'id': 289118612, 'id_str': '289118612', 'name...",In this clip he\n1. Denies WHO's coronavirus d...,"{'hashtags': [{'text': 'coronavirus', 'indices...",2020-04-25,1254024693960540160,None,None,None,None,0,0,0,0,en
1,289118612,TomthunkitsMind,Tomthunkit™,22437,644b36b7a1b284129a82c99b,"{'id': 289118612, 'id_str': '289118612', 'name...",I’ve found the Corona bug.\n\n#coronapocolypse...,"{'hashtags': [{'text': 'coronapocolypse', 'ind...",2020-04-25,1254028436856766466,None,None,None,None,0,0,0,0,en


#### for cached data

In [53]:
def test_cached_performance():
    myquery={}
    search_keyword = input("Please enter a keyword: ")
    search_hashtag = input("Please enter a hashtag: ")
    search_user = input("Please enter a user_name: ")
    start_at = input("Please enter a start date(format:yyyy-mm-dd hh:mm:ss): ")
    end_at = input("Please enter a end date(format:yyyy-mm-dd hh:mm:ss): ")
    # search start
    start_time = time.time()
    # search in NRDB
    # screen name
    if search_user != "":
        # MySQL & Cache: user in cache or not in cache
        if search_user in cache:
            # cache hit: get user data from cache directly
            user_data = pd.DataFrame.from_dict(cache.get(search_user), orient = "index").T
            print("in cache")
        else:
            print("not in cache")
            # cache miss: get data from database
            cur.execute("SELECT userid_str, screen_name, name, favourites_count FROM users WHERE screen_name = %s;", (search_user,))
            user_data = pd.DataFrame(cur.fetchall(),columns=["userid_str", "screen_name", "name", "favourites_count"])
            # Add user data to cache
            user_data_dict = user_data.to_dict("index")[0]
            cache.set(search_user, user_data_dict)
            # del cache.data[user_id]
            # del cache.access_count[user_id]
        # MongoDB
        userid_str = user_data["userid_str"].tolist()
        myquery["user.id_str"] = {"$in":userid_str}
    if search_keyword != "":
        myquery["$text"] = {"$search": search_keyword}
    if search_hashtag != "":
        myquery["entities.hashtags.text"] = search_hashtag
    # time range
    if start_at and end_at != "":
        myquery['created_at'] = {'$gte': start_at, '$lte': end_at}
    elif start_at != "":
        myquery['created_at'] = {'$gte': start_at}
    elif end_at != "":
        myquery['created_at'] = {'$lte': end_at}
    twts = pd.DataFrame(tweets.find(myquery).sort([("favorite_count", pymongo.DESCENDING), ("retweet_count", pymongo.DESCENDING)]).limit(10))
    #Processing
    twts['userid_str'] = pd.json_normalize(twts['user'])['id_str']
    #JOIN
    testItems = pd.merge(user_data, twts, on='userid_str')
    # search end
    end_time = time.time()      
    print(f''' searching in cached information took {end_time - start_time:.4f} seconds''')
    return testItems

In [54]:
test_cached_performance() # first time not in cache, the timing will acturally longer than uncached

Please enter a keyword:  
Please enter a hashtag:  
Please enter a user_name:  TomthunkitsMind
Please enter a start date(format:yyyy-mm-dd hh:mm:ss):  
Please enter a end date(format:yyyy-mm-dd hh:mm:ss):  


not in cache
 searching in cached information took 0.0387 seconds


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,289118612,TomthunkitsMind,Tomthunkit™,22437,644b36b1a1b284129a82bae5,"{'id': 289118612, 'id_str': '289118612', 'name...",In this clip he\n1. Denies WHO's coronavirus d...,"{'hashtags': [{'text': 'coronavirus', 'indices...",2020-04-25,1254024693960540160,None,None,None,None,0,0,0,0,en
1,289118612,TomthunkitsMind,Tomthunkit™,22437,644b36b7a1b284129a82c99b,"{'id': 289118612, 'id_str': '289118612', 'name...",I’ve found the Corona bug.\n\n#coronapocolypse...,"{'hashtags': [{'text': 'coronapocolypse', 'ind...",2020-04-25,1254028436856766466,None,None,None,None,0,0,0,0,en


In [55]:
test_cached_performance() # second time in cache

Please enter a keyword:  
Please enter a hashtag:  
Please enter a user_name:  TomthunkitsMind
Please enter a start date(format:yyyy-mm-dd hh:mm:ss):  
Please enter a end date(format:yyyy-mm-dd hh:mm:ss):  


in cache
 searching in cached information took 0.0346 seconds


,userid_str,screen_name,name,favourites_count,_id,user,text,entities,created_at,id_str,in_reply_to_screen_name,in_reply_to_user_id_str,in_reply_to_status_id_str,quoted_status_id_str,quote_count,reply_count,retweet_count,favorite_count,lang
0,289118612,TomthunkitsMind,Tomthunkit™,22437,644b36b1a1b284129a82bae5,"{'id': 289118612, 'id_str': '289118612', 'name...",In this clip he\n1. Denies WHO's coronavirus d...,"{'hashtags': [{'text': 'coronavirus', 'indices...",2020-04-25,1254024693960540160,None,None,None,None,0,0,0,0,en
1,289118612,TomthunkitsMind,Tomthunkit™,22437,644b36b7a1b284129a82c99b,"{'id': 289118612, 'id_str': '289118612', 'name...",I’ve found the Corona bug.\n\n#coronapocolypse...,"{'hashtags': [{'text': 'coronapocolypse', 'ind...",2020-04-25,1254028436856766466,None,None,None,None,0,0,0,0,en
